In [ ]:
%pip install pandas
%pip install spacy
%pip install fastparquet


In [ ]:
import os
import pandas as pd
import spacy
import subprocess
import sys
import fastparquet


In [ ]:
spacymodel = "en_core_web_lg"
#import spacy
spacymodel = "en_core_web_lg"
# $ python -m spacy download en_core_web_lg

#model = "en_core_web_sm"
model = spacymodel
try:
    spacy.load(model)
    print(f"{model} already installed.")
except OSError:
    print(f"Downloading {model}...")
    subprocess.check_call([sys.executable, "-m", "spacy", "download", model])
    nlp = spacy.load(model)
    print(f"{model} ready: {nlp.pipe_names}")
#print(sp.Default.stop_words)
#nlp = spacy.load(model)
import en_core_web_lg
nlp = en_core_web_lg.load()
doc = nlp("This is a sentence.")
print([(w.text, w.pos_) for w in doc])

model = spacy.load("en_core_web_lg")

In [ ]:
path_cl = "/SPUND/2025/hux/DAIS-C-Annotated - Upload/DAI-C-CL/Speaker Only_Raw"
path_co = "/SPUND/2025/hux/DAIS-C-Annotated - Upload/DAI-C-CO/Speaker Only_Raw"
path_clinical = "DAIS-C-Annotated - Upload/DAI-C-CL/Speaker Only_Raw"
path_control = "DAIS-C-Annotated - Upload/DAI-C-CO/Speaker Only_Raw"
path_parquet = os.environ.get('HKW_TOP') + "/SPUND/2025/hux/"

#path_clinical = os.environ.get('HKW_TOP') + path_cl
#path_control = os.environ.get('HKW_TOP') + path_co
print(os.environ.get('HKW_TOP'))
path_clinical = os.environ.get('HKW_TOP') + path_cl
path_control = os.environ.get('HKW_TOP') + path_co
print(path_clinical)
print(path_control)

In [ ]:
# Collect file names and their content
data = []
for filename in os.listdir(path_clinical):
    if filename.endswith(".txt"):
        file_path = os.path.join(path_clinical, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()
        data.append({"filename": filename, "text": text})

# Convert to DataFrame
clinical = pd.DataFrame(data)



In [ ]:
# Collect file names and their content
data = []
for filename in os.listdir(path_control):
    if filename.endswith(".txt"):
        file_path = os.path.join(path_control, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()
        data.append({"filename": filename, "text": text})

# Convert to DataFrame
control = pd.DataFrame(data)

In [ ]:
control

In [ ]:
clinical

In [ ]:
clinical["group"] = "scz"
control["group"] = "hc"

In [ ]:
clinical

In [ ]:
df = pd.concat([clinical, control], axis=0)

In [ ]:
df = df.replace("\n", " ", regex=True)

In [ ]:
# add custom stopwords
custom_stops = {"er", "erm", "oh", "yeah", "mm", "mhm", "y"}

for w in custom_stops:
    model.Defaults.stop_words.add(w)
    model.vocab[w].is_stop = True


In [ ]:
model.Defaults.stop_words

In [ ]:
def get_cleaned_tokens(text):
    spacy_text = model(text)
    list_of_tokens = [
        token.text
        for token in spacy_text
        if (not token.is_punct)
        and (not token.is_stop)
        and token.text.strip()              # removes whitespace-only tokens
    ]
    return list_of_tokens

def get_chunks(list_of_tokens):
    chunks = [list_of_tokens[i:i+10] for i in range(0, len(list_of_tokens), 10)]
    return chunks


In [ ]:
get_cleaned_tokens("hello world how are you doing")

In [ ]:
df["list_tokens"] = df["text"].apply(get_cleaned_tokens)

In [ ]:
df

In [ ]:
df["list_of_cunks"] = df["list_tokens"].apply(get_chunks)

In [ ]:
df.iloc[0,-1]

In [ ]:
df_exploded = df.explode("list_of_cunks").reset_index(drop=True)

In [ ]:
df_exploded

In [ ]:
df_exploded.to_parquet(path_parquet+"tabelle.parquet")

In [ ]:
text = df.iloc[1,3]

In [ ]:
len(text)

In [ ]:
text

In [ ]:
for element in text:
    print(element)

In [ ]:
#text.split()
# wrong format list

In [ ]:
#sents = [sent.text for sent in model(text).sents]
# wrong format list

In [ ]:
#sents
text_l = df.iloc[1,3]
print(text_l)
import itertools
#text = list(itertools.chain.from_iterable(text_l))
#text = df_exploded[1,-1]
text = ' '.join(text_l)
print(text)

In [ ]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span


def build_nlp():
#    nlp = spacy.load("en_core_web_sm")
    nlp = spacy.load("en_core_web_lg")
    matcher = Matcher(nlp.vocab)

    # -------------------------------------
    # 1. Symptom onset expressions
    # -------------------------------------
    matcher.add(
        "SYMPTOM_ONSET",
        [
            [{"LOWER": "since"}, {"LOWER": {"IN": ["childhood", "infancy", "puberty", "teenage", "teens"]}}],
            [{"LOWER": "since"}, {"LOWER": "i"}, {"LOWER": "was"}, {"IS_DIGIT": True}],
            [{"LOWER": "back"}, {"LOWER": "in"}, {"LOWER": {"IN": ["college", "school", "university", "high", "uni"]}}],
            [{"LOWER": "in"}, {"LOWER": "my"}, {"LOWER": {"IN": ["early", "mid", "late"]}},
             {"LOWER": {"IN": ["teens", "twenties", "thirties"]}}],
        ],
    )

    # -------------------------------------
    # 2. Episode-level markers
    # -------------------------------------
    matcher.add(
        "EPISODE_TEMPORAL",
        [
            [{"LOWER": "last"}, {"LOWER": "episode"}, {"LOWER": "in"}, {"ENT_TYPE": "DATE"}],
            [{"LOWER": "during"}, {"LOWER": "my"},
             {"LOWER": {"IN": ["worst", "last", "previous"]}},
             {"LOWER": {"IN": ["period", "episode"]}}, {"LOWER": "in"}, {"ENT_TYPE": "DATE"}],
        ],
    )

    # -------------------------------------
    # 3. Treatment-related
    # -------------------------------------
    matcher.add(
        "TREATMENT_TEMPORAL",
        [
            [{"LOWER": {"IN": ["started", "began", "quit", "stopped"]}},
             {"LOWER": {"IN": ["meds", "medications", "therapy"]}},
             {"LOWER": "in"}, {"ENT_TYPE": "DATE"}],

            [{"LOWER": {"IN": ["hospitalised", "hospitalized"]}},
             {"IS_DIGIT": True}, {"LOWER": {"IN": ["years", "months", "weeks"]}},
             {"LOWER": "ago"}],
        ],
    )

    # -------------------------------------
    # 4. Relapse
    # -------------------------------------
    matcher.add(
        "RELAPSE_TEMPORAL",
        [
            [{"LOWER": {"IN": ["relapsed", "relapse"]}},
             {"LOWER": "again"}, {"LOWER": {"IN": ["this", "last"]}},
             {"LOWER": {"IN": ["year", "month", "week"]}}],

            [{"LOWER": {"IN": ["episode", "episodes"]}},
             {"LOWER": "last"}, {"LOWER": {"IN": ["month", "year"]}}],
        ],
    )

    # -------------------------------------
    # 5. DURATIONS (corrected)
    # -------------------------------------
    matcher.add(
        "DURATION_TEMPORAL",
        [
            # For six months / for months / for a year
            [
                {"LOWER": "for"},
                {"LOWER": {"IN": ["a", "an"]}, "OP": "?"},
                {"IS_DIGIT": True, "OP": "?"},
                {"LOWER": {"IN": ["day", "days", "week", "weeks", "month",
                                   "months", "year", "years", "decade"]}}
            ],

            # past few months
            [
                {"LOWER": "past"},
                {"LOWER": {"IN": ["few", "couple", "several"]}},
                {"LOWER": {"IN": ["days", "weeks", "months", "years"]}}
            ]
        ],
    )

    # -------------------------------------
    # 6. Life-event anchors
    # -------------------------------------
    matcher.add(
        "LIFE_EVENT_TEMPORAL",
        [
            [{"LOWER": {"IN": ["after", "before", "around", "during"]}},
             {"LOWER": {"IN": ["the", "my"]}, "OP": "?"},
             {"LOWER": {"IN": [
                 "divorce", "breakup", "accident", "pregnancy", "diagnosis",
                 "hospitalisation", "hospitalization", "finals", "lockdown"
             ]}}],
        ],
    )

    # -------------------------------------
    # 7. Calendar markers
    # -------------------------------------
    seasons = ["spring", "summer", "autumn", "fall", "winter"]
    holidays = ["christmas", "easter", "ramadan", "new", "new-year", "new year"]

    matcher.add(
        "CALENDAR_TEMPORAL",
        [
            [{"LOWER": {"IN": ["this", "last"]}}, {"LOWER": {"IN": seasons}}],
            [{"LOWER": {"IN": ["this", "last"]}}, {"LOWER": {"IN": holidays}}],
            [{"LOWER": {"IN": ["earlier", "end"]}}, {"LOWER": "this"},
             {"LOWER": {"IN": ["week", "month", "year"]}}],
        ],
    )

    # -------------------------------------
    # 8. Vague expressions
    # -------------------------------------
    matcher.add(
        "VAGUE_TEMPORAL",
        [
            [{"LOWER": "a"}, {"LOWER": "while"}, {"LOWER": "ago"}],
            [{"LOWER": "ages"}, {"LOWER": "ago"}],
            [{"LOWER": "the"}, {"LOWER": "other"}, {"LOWER": "day"}],
            [{"LOWER": "recently"}],
        ],
    )

    # -------------------------------------
    # 9. Numeric date formats
    # -------------------------------------
    matcher.add(
        "NUMERIC_DATE",
        [
            # 12/2020, 05/21
            [{"SHAPE": "dd"}, {"TEXT": "/"}, {"SHAPE": {"IN": ["dd", "dddd"]}}],
            # 12.05.2020
            [{"SHAPE": "dd"}, {"TEXT": "."}, {"SHAPE": "dd"},
             {"TEXT": ".", "OP": "?"}, {"SHAPE": "dddd", "OP": "?"}],
            # 2020–21, 2019/2020
            [{"SHAPE": "dddd"}, {"TEXT": {"IN": ["–", "-", "/"]}}, {"SHAPE": {"IN": ["dd", "dddd"]}}],
        ],
    )

    # -------------------------------------
    # 10. Diagnosis + date
    # -------------------------------------
    matcher.add(
        "DIAGNOSIS_DATE",
        [
            [{"LOWER": {"IN": ["diagnosed", "dx"]}}, {"LOWER": "in"}, {"ENT_TYPE": "DATE"}],
            [{"LOWER": {"IN": ["diagnosed", "dx"]}}, {"IS_DIGIT": True}],  # diagnosed at 14
            [{"LOWER": {"IN": ["got", "received"]}}, {"LOWER": "my"},
             {"LOWER": "diagnosis"}, {"LOWER": "in"}, {"ENT_TYPE": "DATE"}],
        ],
    )

    nlp.matcher = matcher
    return nlp


def extract_time_spans(text, nlp):
    doc = nlp(text)
    spans = []

    # built-in DATE entities
    for ent in doc.ents:
        if ent.label_ == "DATE":
            spans.append(ent.text)

    # rule-based matches
    for match_id, start, end in nlp.matcher(doc):
        spans.append(doc[start:end].text)

    # dedupe and preserve order
    seen = set()
    final = []
    for s in spans:
        if s not in seen:
            seen.add(s)
            final.append(s)

    return final


if __name__ == "__main__":
    nlp = build_nlp()
    
    # text = """
    # I was diagnosed in 2018 after the accident.
    # Symptoms have been there since childhood and got worse last winter.
    # I relapsed again this year after quitting meds two months ago.
    # Started therapy in June. Hospitalised three years ago.
    # Had another episode last month and during lockdown things escalated.
    # I struggled for six months before the diagnosis.
    
    # """

    print(extract_time_spans(text, nlp))
